## 1 Recognize objects in image (or classify image)

Using trained NN, get object label or labels for image, or otherwise provide a label for the image. Also store the centrality of the object. 

## 2 Generate semantic word families

For each label, use Word2Vec `similar` to retrieve list of words semantically related to the image object labels

## 3 Generate all related words

For each semantically related (below a distance threshold) word to each object label, measure its phonetic similarity to all words in the dictionary. Also store each words's distance.

For each word in each semantic family, sort and choose the phonetically closest (below a distance threshold) words.
(One way is to convert the word to IPA and compare to an IPA converted version of every word in the CMU dictionary.)

## 4 Generate captions

For each word in the phonetic family, of each word in the semantic family, of each object label, retrieve each idiom containing the word.
Add the idiom Id, as well as the stats on the object centrality, semantic distance, and phonetic distance, to a dataframe.

Compute _suitability score_ for each word-idiom match and add this to that column of the dataframe

Also, for each word in the semantic family, search the joke list for match and add that these to a joke_match dataframe, to use if there's too low a suitability score using a substitution.


## 5 Choose captions

Sort captions dataframe by the _suitability score_

Choose the top 10 and generate a list containing each caption with the original semantic family word substituted into the idiom in addition to jokes containing any of the semantic family words

In [6]:
import pandas as pd
from collections import namedtuple

## 1 Recognize objects in image (or classify image)

Using trained NN, get object label or labels for image, or otherwise provide a label for the image. Also store the centrality of the object. 

## 2 Generate semantic word families

For each label, use Word2Vec `similar` to retrieve list of words semantically related to the image object labels

## 3 Generate all related words

For each semantically related (below a distance threshold) word to each object label, measure its phonetic similarity to all words in the dictionary. Also store each words's distance.

For each word in each semantic family, sort and choose the phonetically closest (below a distance threshold) words.
(One way is to convert the word to IPA and compare to an IPA converted version of every word in the CMU dictionary.)

## 4 Generate captions

For each word in the phonetic family, of each word in the semantic family, of each object label, retrieve phrases containing the word.
Add the phrase_Id, as well as the stats on the object centrality, semantic distance, and phonetic distance, to a dataframe.

Compute _suitability score_ for each word-phrase match and add this to that column of the dataframe

Also, for each word in the semantic family, search the joke list for match and add that these to a joke_match dataframe, to use if there's too low a suitability score using a substitution.


In [152]:
import uuid


In [185]:
Phrase = namedtuple('Phrase',['text_string', 'word_list','phon_list','string_length', 'word_count', 'prefix', 'phrase_type'])
phrase_dict = dict()

In [186]:

# toy example of the dict of phrases
t_string = 'Smarter than the average bear'
w_list = t_string.lower().split()
ph_id1 = uuid.uuid1()
phrase_dict[ph_id1] = Phrase(text_string = t_string, word_list = w_list, phon_list = w_list, string_length = len(t_string), word_count = len(w_list), prefix="As usual: ", phrase_type='idiom' )

# toy example of the dict
t_string = 'Not a hair out of place'
w_list = t_string.lower().split()
ph_id2 = uuid.uuid1()
phrase_dict[ph_id2] = Phrase(text_string = t_string, word_list = w_list, phon_list = w_list, string_length = len(t_string), word_count = len(w_list), prefix="As usual: ", phrase_type='idiom' )

# toy example of the dict
t_string = 'Three blind mice'
w_list = t_string.lower().split()
ph_id3 = uuid.uuid1()
phrase_dict[ph_id3] = Phrase(text_string = t_string, word_list = w_list, phon_list = w_list, string_length = len(t_string), word_count = len(w_list), prefix="As usual: ", phrase_type='idiom' )



In [ ]:
phrase_dict[ph_id1].word_list

In [200]:
Close_word = namedtuple('Close_word', ['word', 'distance'])
Phon_family = namedtuple('Phon_family', ['locus_word', 'close_words'])


In [206]:
pair_phon_fam = Phon_family(locus_word='pair', close_words = [Close_word('bear', 1.5), Close_word('hair', 2.7)])
pair_phon_fam

twice_phon_fam = Phon_family(locus_word='twice', close_words = [Close_word('mice', 2.1)])
twice_phon_fam

Phon_family(locus_word='twice', close_words=[Close_word(word='mice', distance=2.1)])

In [188]:


col_names = ['semantic_match', 'sem_dist', 'phonetic_match', 'phon_dist', 'phrase_id', 'score']

cand_df = pd.DataFrame({"semantic_match": ['pair','pair','twice'], "sem_dist": [5, 5, 4.1], "phonetic_match": ['bear', 'hair','mice'], "phon_dist": [1.5, 2.7, 2.1], "phrase_id":  [ph_id1, ph_id2,ph_id3], "phrase_type":  ['idiom', 'idiom','idiom'],'score': [.5, .3, 1.1]})


In [189]:
cand_df['score'] = cand_df.apply(lambda row: 1.0/(row['sem_dist'] + row['phon_dist']), axis=1)

## 5 Choose captions

Sort captions dataframe by the _suitability score_

Choose the top 10(?) and generate a list containing each caption with the original semantic family word substituted into the idiom in addition to jokes containing any of the semantic family words

In [190]:
def sub(phrase, orig_word='', new_word=''):
    return phrase.text_string.replace(orig_word, new_word)

In [193]:
def construct_caption( row ):
    return sub( phrase_dict[ row['phrase_id']], row['phonetic_match'],  row['semantic_match']  )

In [194]:
def get_best_captions(df, selection_size=10):
    df.sort_values(by='score', inplace=True)
    best_df = df.head(selection_size)
    best_df['caption'] = best_df.apply(construct_caption, axis=1 )    
    return best_df
    #return best_df['_caption'].to_list()

In [195]:
best_captions_df = get_best_captions(cand_df)
best_captions_df

<ipython-input-194-0abcd1c06ef8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_df['caption'] = best_df.apply(construct_caption, axis=1 )


,semantic_match,sem_dist,phonetic_match,phon_dist,phrase_id,phrase_type,score,caption
1,pair,5.0,hair,2.7,affde124-7ed1-11eb-ba55-acde48001122,idiom,0.129870,Not a pair out of place
0,pair,5.0,bear,1.5,affdd990-7ed1-11eb-ba55-acde48001122,idiom,0.153846,Smarter than the average pair
2,twice,4.1,mice,2.1,affde7be-7ed1-11eb-ba55-acde48001122,idiom,0.161290,Three blind twice


In [196]:
best_captions_list = best_captions_df['caption'].to_list()
best_captions_list 

['Not a pair out of place',
 'Smarter than the average pair',
 'Three blind twice']